In [1]:
!pip install rake_nltk
#rake_nltk is the library i am  using to extract keywords from description!. it has stopwords (by default english ) preloaded

In [2]:
from rake_nltk import Rake
import nltk
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_csv('/content/drive/MyDrive/bookfinal/book_final_data.csv', engine='python', error_bad_lines=False)
df.head()
df=df[['isbn','book_title','book_author','Summary','Category']]
df.rename(columns = {'Summary':'description', 'Category':'categories'}, inplace = True)
df.drop_duplicates(subset ="book_title",
                     keep = 'last', inplace = True)
df.dropna(inplace = True)
print(df.shape)
df.drop_duplicates(subset=['isbn'])#there aren't any duplicate isbns(there are some anamolies)
print(df.shape)
df['description'].apply(type).value_counts()
import string
df['description'] = df['description'].str.replace('[{}]'.format(string.punctuation), '')
df['description'].astype(str)
df['description'].apply(type).value_counts()



(79583, 5)
(79583, 5)


<class 'str'>    79583
Name: description, dtype: int64

In [3]:
import re
def preprocess_text(text):
    text = re.sub(r'([^\s\w]|_)+', '', text)
    return text
  
df['categories'] = df['categories'].map(preprocess_text)


In [4]:
df['Key_words'] = ''
r = Rake()
for index, row in df.iterrows():
  r.extract_keywords_from_text(row['description'])   # to extract key words, default in lower case
  key_words_dict_scores = r.get_word_degrees()   
  row['Key_words'] = list(key_words_dict_scores.keys())   # to assign list of key words to new column


df.head()


,isbn,book_title,book_author,description,categories,Key_words
1,0002005018,Clara Callan,Richard Bruce Wright,In a small town in Canada Clara Callan relucta...,Actresses,"[sister, nora, bound, small, town, new, york, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Describes the great flu epidemic of 1918 an ou...,Medical,"[describes, great, flu, epidemic, prevent, ano..."
9,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,Essays by respected military historians includ...,History,"[essays, history, differently, worse, respecte..."
10,0671870432,PLEADING GUILTY,Scott Turow,9,9,[9]
66,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,Staring unflinchingly into the abyss of slaver...,Fiction,"[intimate, novel, transforms, history, exodus,..."


In [5]:
#adding the categories i.e. genres to keywords as well so that similar genre books can be selected
df['categories']= df['categories'].apply(lambda x:x.split())
df['categories'].astype(str)
df['Key_words']=df['categories']+df['Key_words']
df.head()


,isbn,book_title,book_author,description,categories,Key_words
1,0002005018,Clara Callan,Richard Bruce Wright,In a small town in Canada Clara Callan relucta...,[Actresses],"[Actresses, sister, nora, bound, small, town, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Describes the great flu epidemic of 1918 an ou...,[Medical],"[Medical, describes, great, flu, epidemic, pre..."
9,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,Essays by respected military historians includ...,[History],"[History, essays, history, differently, worse,..."
10,0671870432,PLEADING GUILTY,Scott Turow,9,[9],"[9, 9]"
66,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,Staring unflinchingly into the abyss of slaver...,[Fiction],"[Fiction, intimate, novel, transforms, history..."


**making authors names into keywords as well, each author's name will be merged(firstnamelastname) and added as a separate keyword. **

In [6]:
df.rename(columns = {'book_author':'authors'}, inplace = True)


In [7]:
df['authors']= df['authors'].apply(lambda x:x.lower().replace(' ',''))

In [8]:
df.head()

,isbn,book_title,authors,description,categories,Key_words
1,0002005018,Clara Callan,richardbrucewright,In a small town in Canada Clara Callan relucta...,[Actresses],"[Actresses, sister, nora, bound, small, town, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,ginabarikolata,Describes the great flu epidemic of 1918 an ou...,[Medical],"[Medical, describes, great, flu, epidemic, pre..."
9,0425176428,What If?: The World's Foremost Military Histor...,robertcowley,Essays by respected military historians includ...,[History],"[History, essays, history, differently, worse,..."
10,0671870432,PLEADING GUILTY,scottturow,9,[9],"[9, 9]"
66,0452264464,Beloved (Plume Contemporary Fiction),tonimorrison,Staring unflinchingly into the abyss of slaver...,[Fiction],"[Fiction, intimate, novel, transforms, history..."


In [9]:
df['authors']= df['authors'].apply(lambda x:x.lower().replace(';',' '))
df.head()

,isbn,book_title,authors,description,categories,Key_words
1,0002005018,Clara Callan,richardbrucewright,In a small town in Canada Clara Callan relucta...,[Actresses],"[Actresses, sister, nora, bound, small, town, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,ginabarikolata,Describes the great flu epidemic of 1918 an ou...,[Medical],"[Medical, describes, great, flu, epidemic, pre..."
9,0425176428,What If?: The World's Foremost Military Histor...,robertcowley,Essays by respected military historians includ...,[History],"[History, essays, history, differently, worse,..."
10,0671870432,PLEADING GUILTY,scottturow,9,[9],"[9, 9]"
66,0452264464,Beloved (Plume Contemporary Fiction),tonimorrison,Staring unflinchingly into the abyss of slaver...,[Fiction],"[Fiction, intimate, novel, transforms, history..."


In [10]:
df

,isbn,book_title,authors,description,categories,Key_words
1,0002005018,Clara Callan,richardbrucewright,In a small town in Canada Clara Callan relucta...,[Actresses],"[Actresses, sister, nora, bound, small, town, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,ginabarikolata,Describes the great flu epidemic of 1918 an ou...,[Medical],"[Medical, describes, great, flu, epidemic, pre..."
9,0425176428,What If?: The World's Foremost Military Histor...,robertcowley,Essays by respected military historians includ...,[History],"[History, essays, history, differently, worse,..."
10,0671870432,PLEADING GUILTY,scottturow,9,[9],"[9, 9]"
66,0452264464,Beloved (Plume Contemporary Fiction),tonimorrison,Staring unflinchingly into the abyss of slaver...,[Fiction],"[Fiction, intimate, novel, transforms, history..."
...,...,...,...,...,...,...
153423,0812533232,Dragon Rigger,"jeffrey,a.carver",9,[9],"[9, 9]"
153424,0812543181,Not This August,cyrilkornbluth,9,[9],"[9, 9]"
153425,0821726595,Mistress of Falcon Court,charlottelammert,Colin O39Bannion39s dying request places his d...,[Fiction],"[Fiction, daughter, molly, dark, secrets, surr..."
153426,155882068X,The Balloon Man (Crime Classics),charlottearmstrong,9,[9],"[9, 9]"


In [11]:
df['authors']= df['authors'].apply(lambda x:x.split())
df['authors'].astype(str)
df['Key_words']=df['authors']+df['Key_words']
df.head()


,isbn,book_title,authors,description,categories,Key_words
1,0002005018,Clara Callan,[richardbrucewright],In a small town in Canada Clara Callan relucta...,[Actresses],"[richardbrucewright, Actresses, sister, nora, ..."
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,[ginabarikolata],Describes the great flu epidemic of 1918 an ou...,[Medical],"[ginabarikolata, Medical, describes, great, fl..."
9,0425176428,What If?: The World's Foremost Military Histor...,[robertcowley],Essays by respected military historians includ...,[History],"[robertcowley, History, essays, history, diffe..."
10,0671870432,PLEADING GUILTY,[scottturow],9,[9],"[scottturow, 9, 9]"
66,0452264464,Beloved (Plume Contemporary Fiction),[tonimorrison],Staring unflinchingly into the abyss of slaver...,[Fiction],"[tonimorrison, Fiction, intimate, novel, trans..."


In [12]:
#creating the bag of words which is used by the rake function to get vectors
df['Bag_of_words'] = ''
for index, row in df.iterrows():
    words = ''
    words += ' '.join(row['Key_words']) + ' '
    row['Bag_of_words'] = words
df['Bag_of_words'] = df['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')


In [13]:
df.head(7)

,isbn,book_title,authors,description,categories,Key_words,Bag_of_words
1,0002005018,Clara Callan,[richardbrucewright],In a small town in Canada Clara Callan relucta...,[Actresses],"[richardbrucewright, Actresses, sister, nora, ...",richardbrucewright Actresses sister nora bound...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,[ginabarikolata],Describes the great flu epidemic of 1918 an ou...,[Medical],"[ginabarikolata, Medical, describes, great, fl...",ginabarikolata Medical describes great flu epi...
9,0425176428,What If?: The World's Foremost Military Histor...,[robertcowley],Essays by respected military historians includ...,[History],"[robertcowley, History, essays, history, diffe...",robertcowley History essays history differentl...
10,0671870432,PLEADING GUILTY,[scottturow],9,[9],"[scottturow, 9, 9]",scottturow 9 9
66,0452264464,Beloved (Plume Contemporary Fiction),[tonimorrison],Staring unflinchingly into the abyss of slaver...,[Fiction],"[tonimorrison, Fiction, intimate, novel, trans...",tonimorrison Fiction intimate novel transforms...
75,0609804618,Our Dumb Century: The Onion Presents 100 Years...,[theonion],The staff of The Onion presents a satirical co...,[Humor],"[theonion, Humor, account, staff, satirical, c...",theonion Humor account staff satirical collect...
77,1841721522,New Vegetarian: Bold and Beautiful Recipes for...,[celiabrooksbrown],In New Vegetarian Celia Brooks Brown presents ...,[Cooking],"[celiabrooksbrown, Cooking, innovative, approa...",celiabrooksbrown Cooking innovative approach n...


In [14]:
#trial
df['Bag_of_words'][3]


'ginabarikolata Medical describes great flu epidemic prevent another lethal pandemic understand forty million people worldwide discusses efforts outbreak public health officials scientists 1918 killed'

In [15]:
df=df.iloc[0:9000].reset_index()


In [16]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Bag_of_words'])
count_matrix


<9000x27291 sparse matrix of type '<class 'numpy.int64'>'
	with 110113 stored elements in Compressed Sparse Row format>

In [17]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

[[1.         0.         0.         ... 0.         0.         0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.20412415 0.        ]
 [0.         0.         0.         ... 0.20412415 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [18]:
indices = pd.Series(df['isbn'])
indices[:5]


0    0002005018
1    0374157065
2    0425176428
3    0671870432
4    0452264464
Name: isbn, dtype: object

In [19]:
df

,index,isbn,book_title,authors,description,categories,Key_words,Bag_of_words
0,1,0002005018,Clara Callan,[richardbrucewright],In a small town in Canada Clara Callan relucta...,[Actresses],"[richardbrucewright, Actresses, sister, nora, ...",richardbrucewright Actresses sister nora bound...
1,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,[ginabarikolata],Describes the great flu epidemic of 1918 an ou...,[Medical],"[ginabarikolata, Medical, describes, great, fl...",ginabarikolata Medical describes great flu epi...
2,9,0425176428,What If?: The World's Foremost Military Histor...,[robertcowley],Essays by respected military historians includ...,[History],"[robertcowley, History, essays, history, diffe...",robertcowley History essays history differentl...
3,10,0671870432,PLEADING GUILTY,[scottturow],9,[9],"[scottturow, 9, 9]",scottturow 9 9
4,66,0452264464,Beloved (Plume Contemporary Fiction),[tonimorrison],Staring unflinchingly into the abyss of slaver...,[Fiction],"[tonimorrison, Fiction, intimate, novel, trans...",tonimorrison Fiction intimate novel transforms...
...,...,...,...,...,...,...,...,...
8995,51338,1591161932,Flame Of Recca (Flame Of Recca),[nobuyukianzai],Armed with the power to control flame Recca Ha...,"[Comics, Graphic, Novels]","[nobuyukianzai, Comics, Graphic, Novels, awkwa...",nobuyukianzai Comics Graphic Novels awkward si...
8996,51340,1591820146,Love Hina (Book 3),[kenakamatsu],9,[9],"[kenakamatsu, 9, 9]",kenakamatsu 9 9
8997,51342,1591820200,"Real Bout High School, Book 3",[reijisaiga],9,[9],"[reijisaiga, 9, 9]",reijisaiga 9 9
8998,51344,1591820219,"Real Bout High School, Book 4",[reijisaiga],Ryoko Mitsurugi the Samurai Girl is the top fi...,"[Comics, Graphic, Novels]","[reijisaiga, Comics, Graphic, Novels, martial,...",reijisaiga Comics Graphic Novels martial arts ...


**we are searching the book by isbn10 here but in the final code will use fuzzywuzzy to search through entering book title like done in the colab_filter **

In [27]:
def recommend(isbn, cosine_sim = cosine_sim):
    recommended_books = []
    resdf=pd.DataFrame()
    idx = indices[indices == isbn].index[0]   # to get the index of the movie title matching the input movie
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)  
    top_10_indices = list(score_series.iloc[1:11].index)   # to get the indices of top 10 most similar movies
    # [1:11] to exclude 0 (index 0 is the input movie itself)
    for i in top_10_indices:   # to append the titles of top 10 similar movies to the recommended_movies list
        recommended_books.append(list(df['isbn'])[i])
    resdf=df[df['isbn'].isin(recommended_books)]  
    return resdf

recommend('0002005018')# try 0439064864 for harry potter 2, 0439064864 is the time machine by hg wells

#df.loc[df['isbn10'] == '0451528557']
#print(df['title'][2510])#so duplicate book titles exist!

,index,isbn,book_title,authors,description,categories,Key_words,Bag_of_words
1126,14730,0060977507,The Beet Queen : A Novel,[louiseerdrich],In the early 1930s Karl and his sister Mary Ad...,[Fiction],"[louiseerdrich, Fiction, refuge, sister, mary,...",louiseerdrich Fiction refuge sister mary adare...
1797,18762,1550223968,Too Close to the Falls: A Memoir,[catherinegildiner],The author shares her memories that are by tur...,"[Biography, Autobiography]","[catherinegildiner, Biography, Autobiography, ...",catherinegildiner Biography Autobiography auth...
3123,26262,0882893327,The Voodoo Queen: A Novel (Pelican Pouch Series),[roberttallant],Reprint Originally published New York Putnam c...,"[Biography, Autobiography]","[roberttallant, Biography, Autobiography, repr...",roberttallant Biography Autobiography reprint ...
3451,27764,1931561230,Sleep Toward Heaven: A Novel,[amandaeyreward],The lives of three womena twentynine yearold d...,[Fiction],"[amandaeyreward, Fiction, lives, recently, wid...",amandaeyreward Fiction lives recently widowed ...
4238,31184,2266102621,Les Heures / The Hours,[michaelcunningham],C39est à New York à la fin du XXe siècle,[Fiction],"[michaelcunningham, Fiction, c39est, à, new, y...",michaelcunningham Fiction c39est à new york la...
5077,35805,055328942X,"The Shelters of Stone (Earth's Children, Book 5)",[jeanm.auel],Originally published New York Crown 2002,[Fiction],"[jeanm.auel, Fiction, originally, published, n...",jeanm.auel Fiction originally published new yo...
5393,37198,0373825013,"Whirlwind (Tyler, Book 1)",[nancymartin],The first in a new romance series introduces r...,[Fiction],"[nancymartin, Fiction, important, part, new, r...",nancymartin Fiction important part new romance...
5458,37497,0060958111,Extra Virgin: A Young Woman Discovers the Ital...,[anniehawes],In 1983 a pale Annie Hawes and her equally pal...,[Travel],"[anniehawes, Travel, small, italian, town, equ...",anniehawes Travel small italian town equally p...
6808,43561,0553293885,A Great Day for the Deadly,[janehaddam],On the eve of St Patrick39s Day in a small ups...,[Fiction],"[janehaddam, Fiction, young, nun, small, upsta...",janehaddam Fiction young nun small upstate new...
7174,44920,1853265535,Washington Square (Classics Library (NTC)),[henryjames],In the Washington Square area of New York City...,[Fiction],"[henryjames, Fiction, father, fragile, catheri...",henryjames Fiction father fragile catherine sl...


**conclusion**
we can see all of these are similar movies, share categories and from the description we can tell that they are science fiction books and have the author H.G. wells in common in one some too. 